In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Lab 2023/Lab 5

This is an example of using RNN and LSTM to predict the next value in a time series.

In [ ]:
# Import pandas and give it an alias pd
import pandas as pd

In [ ]:
dataframe = pd.read_csv('airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [ ]:
# plot dataset and required packages
import matplotlib.pyplot as plt

plt.plot(dataset)
plt.show()

In [ ]:
# normalize the dataset and import required packages
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# split the dataset into train and test sets ( maintaining the temporal order of the data)
train_ratio = 0.67
train_size = int(len(dataset) * train_ratio)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
'''
Task: After you run this notebook successfuly, change train_ratio to be 0.80, and run the notebook again, record the evaluation metrics and then compare the results
'''

In [ ]:
# print the sizes of train and test sets
print(len(train), len(test))

In [ ]:
# The create_dataset function is used to transform a given dataset into a format that's suitable for time series forecasting.
# The look_back parameter specifies how many previous steps to use as input features for the prediction at the next time step.
# For example, when we have a dataset = [[1], [2], [3], [4], [5], [6]] and look_back is 1,
# dataX will be [[1], [2], [3], [4], [5]]
# dataY will be [2, 3, 4, 5, 6]
# So the model will be trained to predict 2 given [1], 3 given [2], 4 given [3], 5 given [4], and 6 given [5].

# import the required package
import numpy as np

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        b = dataset[i + look_back, 0]
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
for i in range (len(trainX)):
    print(trainX[i], trainY[i])

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

Input a model code, `model_code = 1` for RNN and `model_code = 2` for LSTM.

In [ ]:
 model_code = int(input("Input a model code, model_code = 1 for RNN and model_code = 2 for LSTM. Your choice: "))

In [ ]:
# Set up parameters for running the model
epochNo = 100
batchSize = 1

In [ ]:
# create and fit the RNN or LSTM network
# import required packages
import tensorflow as tf
from tensorflow.keras.layers import Dense

if (model_code == 1):
    '''
    Task: import SimpleRNN from tensorflow.keras.layers for using RNN model
    '''
    "Add your code here"
    
    # Answer
    from tensorflow.keras.layers import SimpleRNN


    model = tf.keras.Sequential()
    '''
    Task: Add a SimpleRNN layer using method add() with parameters: (4,input_shape=(1, look_back)) for the SimpleRNN layer
    '''
    "Add your code here"
    
    # Answer
    model.add(SimpleRNN(4,input_shape=(1, look_back)))


    model.add(Dense(1))
    '''
    Task: Compile the model using method compile() from the model with parameters: loss='mean_squared_error', optimizer='adam'
    '''
    "Add your code here"
    
    # Answer
    model.compile(loss='mean_squared_error', optimizer='adam')


    '''
    Task: Train the model using method fit() from the model with the tran dataset, epochs=epochNo, batch_size=batchSize, verbose=2
    '''
    "Add your code here"
    
    # Answer
    model.fit(trainX, trainY, epochs=epochNo, batch_size=batchSize, verbose=2)

elif (model_code == 2):
    '''
    Task: import LSTM from tensorflow.keras.layers for using LSTM model
    '''
    "Add your code here"
    
    # Answer
    from tensorflow.keras.layers import LSTM


    model =tf.keras.Sequential()
    '''
    Task: Add a LSTM layer using method add() with parameters: (4, input_shape=(1, look_back)) for the LSTM layer
    '''
    "Add your code here"
    
    # Answer
    model.add(LSTM(4, input_shape=(1, look_back)))


    model.add(Dense(1))
    
    '''
    Task: Compile the model using method compile() from the model with parameters: loss='mean_squared_error', optimizer='adam'
    '''
    "Add your code here"
    
    # Answer
    model.compile(loss='mean_squared_error', optimizer='adam')
  
    '''
    Task: Train the model using method fit() from the model with the tran dataset, epochs=epochNo, batch_size=batchSize, verbose=2
    '''
    "Add your code here"
    
    # ANswer
    model.fit(trainX, trainY, epochs=epochNo, batch_size=batchSize, verbose=2)

else:
    print("invalid model code, try a correct code, 1 or 2")

In [ ]:
# make predictions of train set
trainPredict = model.predict(trainX)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])

In [ ]:
# make predictions of test set
testPredict = model.predict(testX)

In [ ]:
# invert predictions
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# calculate root mean squared error and import required package
from sklearn.metrics import mean_squared_error

trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()